# eLabFTW - API workshop - Interact using python

## eLabFTW's Python Library: `elabapi-python`

description python sympa
Python is a programming language 

We will now send requests to the API using python, and more precisely a library that has been designed for eLabFTW : elabapi_python.

This library provides a structured approach to sending API requests, following a defined schema. All available methods and their usage are documented in the official API reference.

## Setup the Environment

To ensure a clean and isolated workspace, we will set up a **Python virtual environment** and install the required library using `pip`, as per the documentation suggests.

We are going to create a virtual environment and activate it with the following commands:

In [36]:
# create environment named 'venv'
!python -m venv venv
# activate it
!source elab/bin/activate
# install required library
!pip install elabapi-python

Defaulting to user installation because normal site-packages is not writeable


After installation, your environment is ready to interact with the eLabFTW API. 🚀

📖 **API Documentation:**

🔗 **[doc-elabftw-elabapi](https://doc.elabftw.net/api/elabapi-html/)**

Additionally, example scripts tailored for users can be found in the **examples** folder of the GitHub repository:


📂 **Example Scripts:**

🔗 **[Examples using elabapi-python](https://github.com/elabftw/elabapi-python/tree/master/examples)**

### Sending a Request to the API

To start, we will send a request to the **Info** endpoint, similar to the first example using `curl`. The response should match the output from our earlier `curl` request, confirming that our API authentication and setup are working correctly.

## 🔍 Retrieving Environment Variables

In Python, you can get environment variables by using `os` module. This allows your script to access sensitive variables (such as the key) without having to write them in the script and prevent accidental leakage.

In [43]:
%env API_URL=https://elab.local:3148/api/v2
%env API_KEY=apiKey4Test

env: API_URL=https://elab.local:3148/api/v2
env: API_KEY=apiKey4Test


In [44]:
import os

Let us verify that the correct parameters are present:

In [45]:
print(os.getenv("API_URL"))
print(os.getenv("API_KEY"))

https://elab.local:3148/api/v2
apiKey4Test


## Configuring the eLabAPI-Python Client  

The following code sets up the `elabapi-python` client by providing the necessary API key, host URL, and security settings.



In [49]:
import elabapi_python
# initiate a configuration object
configuration = elabapi_python.Configuration()
# configure API Key
configuration.api_key["api_key"] = os.getenv("API_KEY")
configuration.api_key_prefix["api_key"] = "Authorization"
# configure API Url
configuration.host = os.getenv("API_URL")
#TODO: remove when url corrected
configuration.verify_ssl = False

Initiate the API Client object with the configuration object.

In [50]:
api_client = elabapi_python.ApiClient(configuration)
api_client.set_default_header(header_name="Authorization", header_value=os.getenv("API_KEY"))

We are now done with the library's configuration. This part has to be included in all scripts.

### Info Endpoint

As a first example, we will now use a specific endpoint: `info`.

In [58]:
# instanciate the infoApi object
info_api = elabapi_python.InfoApi(api_client)
# make a get request
api_response = info_api.get_info()
# show the response
print(api_response)

{'active_users_count': 9,
 'all_users_count': 10,
 'elabftw_version': '5.2.0-alpha',
 'elabftw_version_int': 50200,
 'experiments_count': 334,
 'experiments_timestamped_count': 0,
 'items_count': 232,
 'teams_count': 4,
 'ts_balance': 0,
 'uploads_filesize_sum': 2079,
 'uploads_filesize_sum_formatted': '2.03 KiB'}


/usr/lib/python3.13/site-packages/urllib3/connectionpool.py:1064: InsecureRequestWarning: Unverified HTTPS request is being made to host 'elab.local'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


And the response you should get back from the server:

*Formatted JSON:*
```json
{
    'active_users_count': 9,
    'all_users_count': 10,
    'elabftw_version': "5.2.0-alpha",
    'elabftw_version_int': 50200,
    'experiments_count': 334,
    'experiments_timestamped_count': 0,
    'items_count': 232,
    'teams_count': 4,
    'ts_balance': 0,
    'uploads_filesize_sum': 2079,
    'uploads_filesize_sum_formatted': "2.03 KiB"
}
```

## post exp 

description of following exec


In [59]:
experimentsApi = elabapi_python.ExperimentsApi(api_client)

body={'title':'hi'} 

api_response = experimentsApi.post_experiment(body=body)
print(api_response)

/usr/lib/python3.13/site-packages/urllib3/connectionpool.py:1064: InsecureRequestWarning: Unverified HTTPS request is being made to host 'elab.local'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


None


# patch exp


# post fichier


# patch ficher


# get le fchier


# delete